## I Selection des données

### I.1 Selection par ligne

In [1]:
df = df[df['sector'] != "Student"]

NameError: name 'df' is not defined

### I.2 Sélection par colonne

In [ ]:
key_cols = ['SalaryUSD','Country','PostalCode','EmploymentStatus','JobTitle','ManageStaff','YearsWithThisTypeOfJob',
          'HowManyCompanies','OtherPeopleOnYourTeam', 'CompanyEmployeesOverall',"HoursWorkedPerWeek",
          'PopulationOfLargestCityWithin20Miles','EmploymentSector', 'LookingForAnotherJob', 'CareerPlansThisYear',
          'Gender', 'OtherJobDuties']

df = df[key_cols]



Renomage des colonnes

In [ ]:
names_cols = {
    "Country": "location",
    "SalaryUSD": "salary",
    "JobTitle": "job_title",
    "YearsWithThisTypeOfJob": "experience",
    "HowManyCompanies": "num_of_compagnies",
    "OtherPeopleOnYourTeam": "team_size",
    "HoursWorkedPerWeek": "work_hours_per_week",
    "EmploymentStatus": "status",
    "LookingForAnotherJob": "lf_jobs",
    "PostalCode": "postal_code",
    "ManageStaff": "manage",
    "CompanyEmployeesOverall": "num_of_employees",
    "PopulationOfLargestCityWithin20Miles": "city_population",
    "EmploymentSector" : "sector",
    "CareerPlansThisYear": "career_plans",
    "OtherJobDuties": "other_job_duties"   
}

rename_cols(df, names_cols)

## II Traitement des Doublons

### II.1 Recherche des doublons

In [ ]:
print(df1.duplicated().value_counts())

### II.2 Traitement des doublons

Warning: Si vous recherchez des doublons dans une table réduite, vous devez vérifier que ces instances étaient des doublons sur la base brute. La question à se poser c'est "est ce une erreur et les deux lignes correspondent au même individu ou est-ce deux individus différents qui ont exactement les mêmes données et que je dois donc concerver tous les deux dans la base?"

In [ ]:
df.drop_duplicates(subset=df.columns.difference(['Timestamp']), inplace = True)

## III Traitement des valeurs manquantes

### 1. Recherche des valeurs manquantes

In [ ]:
#recherche des valeurs manquantes:

print(df1.isnull().sum()) 
print(df.isna().sum())
print(df1.shape)

### 2. Traitement des valeurs manquantes

La question à se poser est "Que faire de mes données manquantes"

1. La variable pour laquelle il me manque une valeur est centrale pour mon étude (type variable à expliquée), l'ensemble de ma ligne est donc inutile

In [ ]:
# remove all the rows that contain a missing value
nfl_data.dropna()

2. J'ai une variable, peu importante auquelle il manque une grande partie des valeurs, elle est donc inutilisable. Je choisis de la supprimer.

In [ ]:
# remove all columns with at least one missing value
columns_with_na_dropped = nfl_data.dropna(axis=1)

3. Ma variable est importante et je veux pouvoir faire des calculs dessus (empéché par les Nan), je remplace par une valeur (0 ou la moyenne)

In [ ]:
# replace all NA's with 0
subset_nfl_data.fillna(0)

4. Ma variable est importante et ordonnés, je remplace la valeur par sa précédente.

In [ ]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the remaining na's with 0
subset_nfl_data.fillna(method='bfill', axis=0).fillna(0)

5. Ma variable est importante et je n'ai pas besoin d'y faire des calculs, je conserve les valeurs nulles.

## IV Assigner le bon type aux données

### IV.1 Exploration des types

In [ ]:
df.dtypes

### IV.2 Traitement des types

In [ ]:
# create a new column, date_parsed, with the parsed dates
df['date_parsed'] = pd.to_datetime(df['date'], format="%m/%d/%y")

# Transformation d'un varchar en float
df['team_size'] = df['team_size'].astype("float")

# Récupération de la partie numérique d'une variable type texte à l'aide d'une expression régulière
df['min_salary']=df['salary'].str.extract(r'^[^\d]*(\d+)').astype("int32")*1000

## V Consistance, reformatage et valeurs abbérantes.

### V.1 Evaluer la consistance des variables de type caractère.

#### Afficher l'ensemble des valeurs uniques

In [ ]:
print(sorted(df['Column Name'].unique()))
df1["JobTitle"].value_counts()

#### Traitement de la consistence

In [ ]:
# convert to lower case
professors['Country'] = professors['Country'].str.lower()
# remove trailing white spaces
professors['Country'] = professors['Country'].str.strip()

#### Modifier les valeurs peu représentées d'une colonne

L'interet d'avoir moins de catégories est d'avoir des visualisations plus propres et des catégories plus consistantes en vue du machine learning. 

La question à se poser: "Est ce que cette catégorie est suffisament différente des autres et représentée au sein de mon échantillon pour justifier d'être isolée?"

##### Traitement à partir des valeurs

In [ ]:
# utiliser une fonction lambda à une condition
df1["JobTitle"] = df1["JobTitle"].apply(
    lambda x: "Database Admin" if x.startswith('DBA') else x
)

# ou à plusieurs

df1["HowManyCompanies"] = df1["HowManyCompanies"].apply(
    lambda x: "1" if str(x).startswith('1') else "2" if str(x).startswith('2') else x
)

In [ ]:
# autre possibilité avec contains
df.loc[(df["job_title"].str.contains("DBA")), "job_title"] = "DBA"


In [ ]:
# utilisation de "isin"
df1.loc[~df1["Gender"].isin(["Male", "Female"]), "Gender"] = "Other - NA"
# le tilde ~ permet de prendre l'opposé de la condition

##### Traitement à partir du nombre d'occurence

In [ ]:
s = df1["JobTitle"].value_counts()
df1.loc[df1["JobTitle"].isin(s[(s < 5)].index), "JobTitle"] = "Other"

### V.2 Traitements des valeurs abbérantes des variables numériques

Les valeurs abbérantes sont de deux types, elles sont soient 
- impossibles donc fausses et à supprimer
- extrêmes donc génantes et à traiter

La question à se pose est 'auquel de ces deux types correspond ma valeur abbérante"

In [ ]:
#### Afficher des statistiques sur une variable

In [ ]:
df1["SalaryUSD"].describe()

#### Afficher la distribution d'une variable numérique

In [ ]:
plt.figure(figsize = (20,10))
sns.boxenplot(x = df1["SalaryUSD"], k_depth="proportion")

#### Traiter les valeurs impossibles

In [ ]:
# Si la variable est au coeur de l'étude, ou si la fausseté à cette question 
# met en doute l'intégralité de l'instance, on peut supprimer celle-ci
df1 = df1.drop(df1[df1["SalaryUSD"] < 25000].index)

# On peut choisir de supprimer la valeur mais de conserver l'instance.
import numpy as np
df1.loc[df1["SalaryUSD"] < 25000, "SalaryUSD"] = None

# On peut également conserver l'instance 
# en modifiant la valeur pour la remplacer par une valeur standard

df1.loc[df1["SalaryUSD"] < 25000, "SalaryUSD"] = df1['SalaryUSD'].mean()

#### Traiter les valeurs extrèmes

In [ ]:
# En fonction du problème traité, il peut être génant d'avoir des valeurs extrêmes
# Il est possible dans ce cas de les plafonner. 

df1.loc[df1["SalaryUSD"] > 500000, "SalaryUSD"] = 500000

"""Le plafond doit être choisi avec attention, 
    - soit en fonction d'informations métiers
    - soit au niveau d'une rupture de continuité dans la distribution de la variable
"""